In [55]:
import numpy as np
import pandas as pd
from sim_run_obj import SimulationRun

In [56]:
'''
param list satellite
payload_perc
payload_del
bus_perc
bus_del
space_perc
space_del
nlt_threshold
'''

'\nparam list satellite\npayload_perc\npayload_del\nbus_perc\nbus_del\nspace_perc\nspace_del\nnlt_threshold\n'

In [57]:
#last parts are nlt vals
threhold = 245

In [58]:
sat_params = [
            [0.75, 14, 21, 28, 0.1, 7, 14, 21, 0.5, 7, 14, 21, 242],
            [0.5, 7, 14, 21, 0.1, 7, 14, 21, 0.4, 7, 14, 21, 179],
            [0.4, 3, 10, 17, 0.1, 7, 14, 21, 0.3, 7, 14, 21, 234],
            [0.3, 0, 7, 14, 0.1, 7, 14, 21, 0.2, 7, 14, 21, 204],
            [0.2, 0, 3, 10, 0.1, 7, 14, 21, 0.2, 7, 14, 21, 218],
            [0.2, 0, 1, 7, 0.1, 7, 14, 21, 0.2, 7, 14, 21, 232],
            [0.2, 0, 1, 4, 0.1, 7, 14, 21, 0.2, 7, 14, 21, 246],
            [0.2, 0, 1, 2, 0.1, 7, 14, 21, 0.2, 7, 14, 21, 260],
            [0.2, 0, 1, 2, 0.1, 7, 14, 21, 0.2, 7, 14, 21, 274],
            [0.2, 0, 1, 2, 0.1, 7, 14, 21, 0.2, 7, 14, 21, 288]
        ]

In [59]:
'''
param list scenario
ride_share_perc
ride_share_del
launch_one_perc
launch_one_del
launch_two_perc
launch_two_del
initialization_perc_one
initialization_del_one
initialization_perc_two
initialization_del_two
'''

'\nparam list scenario\nride_share_perc\nride_share_del\nlaunch_one_perc\nlaunch_one_del\nlaunch_two_perc\nlaunch_two_del\ninitialization_perc_one\ninitialization_del_one\ninitialization_perc_two\ninitialization_del_two\n'

In [60]:
'''
event logger
launch
satellite
margin against 245
'''

'\nevent logger\nlaunch\nsatellite\nmargin against 245\n'

In [61]:
scen_params = [0.1, 14, 21, 28, 0.1, 7, 14, 21, 0.5, 90, 91, 92, 0.3, 7, 14, 28, 0.1, 6, 7, 14]

In [62]:
nlt_thresh = 245
num_trials = 20000

In [63]:
def run_trials(num_trials,const_params,scenario_params,nlt_thresh):
    above_below_run = []
    sat_delays_run = []
    event_delays_run = []
    scenario_results_run = []
    sim = SimulationRun(const_params,scenario_params,nlt_thresh)
    
    for i in range(num_trials):
        above_below,sat_delays,event_delays,scenario_results = sim.run_sim()
        above_below_run.append(above_below)
        sat_delays_run.append(sat_delays)
        event_delays_run.append(event_delays)
        scenario_results_run.append(scenario_results)
                               
        
    return above_below_run, sat_delays_run, event_delays_run,scenario_results_run

In [64]:
above_below_run, sat_delays_run, event_delays_run,scenario_results = run_trials(num_trials,sat_params,scen_params,nlt_thresh)

In [65]:
def agg_data(array_of_data,method):
    if array_of_data == sat_delays_run:
        sat_df = pd.DataFrame(array_of_data) -245
    else:
        sat_df = pd.DataFrame(array_of_data)
    sat_df.columns = ['sat_'+str(i) for i in range(1,len(sat_df.columns)+1)]
    agg_set = sat_df.agg(method)
    return agg_set

In [66]:
def percentiles(array_of_data,percentile):
    if array_of_data == sat_delays_run:
        df = pd.DataFrame(array_of_data)-245
    else:
        df = pd.DataFrame(array_of_data)
    df.columns = ['sat_'+str(i) for i in range(1,len(df.columns)+1)]
    return df.quantile(percentile)

In [67]:
agg_data(sat_delays_run,'mean')

sat_1     20.51795
sat_2    -52.45995
sat_3     -1.90870
sat_4    -34.95770
sat_5    -22.18265
sat_6     -8.44655
sat_7      5.32705
sat_8     19.08065
sat_9     33.14835
sat_10    47.12095
dtype: float64

In [68]:
agg_data(event_delays_run,'mean')

sat_1     43.68740
sat_2      8.22460
sat_3     23.39590
sat_4      8.24855
sat_5      8.59315
sat_6     14.89180
sat_7     48.02430
sat_8     48.02430
sat_9     48.02430
sat_10    48.02430
dtype: float64

In [69]:
agg_data(scenario_results,'mean')

sat_1     8.2246
sat_2    48.0243
dtype: float64

In [70]:
agg_data(above_below_run,'sum')/num_trials

sat_1     0.11125
sat_2     1.00000
sat_3     0.61935
sat_4     0.99955
sat_5     0.99010
sat_6     0.83350
sat_7     0.00000
sat_8     0.00000
sat_9     0.00000
sat_10    0.00000
dtype: float64

In [71]:
data_dict = {'sat_delay':sat_delays_run,'scenario_results':event_delays_run,'launches':scenario_results}
for k,j in zip(data_dict.keys(),data_dict.values()):
    data_list = []
    for i in [.8,.85,.9,.95]:
        pp = percentiles(j,i)
        data_list.append(pp)
    df = pd.concat(data_list,axis=1)
    print(df)
    df.to_csv(f'data/{k}_results.csv')

        0.80  0.85  0.90  0.95
sat_1   32.0  33.0  35.0  38.0
sat_2  -42.0 -40.0 -37.0 -34.0
sat_3    5.0   8.0  11.0  15.0
sat_4  -28.0 -26.0 -24.0 -20.0
sat_5  -15.0 -13.0 -11.0  -9.0
sat_6   -1.0   1.0   3.0   5.0
sat_7   13.0  15.0  16.0  18.0
sat_8   27.0  28.0  30.0  32.0
sat_9   41.0  43.0  44.0  46.0
sat_10  55.0  57.0  58.0  60.0
        0.80  0.85  0.90   0.95
sat_1   91.0  91.0  91.0  102.0
sat_2   18.0  20.0  22.0   28.0
sat_3   38.0  90.0  91.0   91.0
sat_4   18.0  20.0  22.0   28.0
sat_5   18.0  20.0  23.0   30.0
sat_6   21.0  24.0  37.0   91.0
sat_7   91.0  91.0  91.0  105.0
sat_8   91.0  91.0  91.0  105.0
sat_9   91.0  91.0  91.0  105.0
sat_10  91.0  91.0  91.0  105.0
       0.80  0.85  0.90   0.95
sat_1  18.0  20.0  22.0   28.0
sat_2  91.0  91.0  91.0  105.0
